# result

> Conc result class.
- toc: false
- page-layout: full

In [ ]:
#| default_exp result

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import polars as pl
from fastcore.basics import patch
from great_tables import GT

In [ ]:
#| export
class Result:
	def __init__(self,
			  type: str, # report type
			  df: pl.DataFrame|pl.LazyFrame, # Polars dataframe or lazyframe with the results
			  title: str, # title of the report
			  description: str, # description
			  summary_data: dict, # summary data (ignored)
			  formatted_data: list[str] # list of formatted data about the table
			  ): 

		""" Class for results from Conc reports """
		self.type = type
		self.df = df
		self.title = title
		self.description = description
		self.summary_data = summary_data
		self.formatted_data = formatted_data
		


In [ ]:
#| export
@patch
def to_frame(self: Result,
			 collect_if_lazy: bool = True # if the df is a lazyframe, collect before returning
			 ):
	""" Return result output from conc as a dataframe """
	if collect_if_lazy and type(self.df) == pl.LazyFrame:
		self.df = self.df.collect()
	return self.df

In [ ]:
#| exporti
@patch
def _prepare(self:Result
			 ):
	""" Prepare the result for display """

	if hasattr(self, '_gt'):
		pass
	else:
		columns_with_integers = []
		columns_with_decimals = []
		
		self.df = self.to_frame(collect_if_lazy=True)

		if self.df.select(pl.len()).item() > 0:

			self.df.columns = [col.replace('_', ' ').title() for col in self.df.columns]

			for col in self.df.columns:
				if self.df[col].dtype in [pl.Float64, pl.Float32]:
					columns_with_decimals.append(col)
				elif not col.endswith('Id') and self.df[col].dtype in [pl.Int64, pl.Int32, pl.Int16, pl.Int8, pl.UInt64, pl.UInt32, pl.UInt16, pl.UInt8]:
					columns_with_integers.append(col)

		self._gt = GT(self.df).tab_options(table_margin_left = 0)
		if self.title != '' or self.description != '':
			self._gt = self._gt.tab_header(self.title, self.description)

		if len(columns_with_decimals) > 0:
			self._gt = self._gt.fmt_number(columns_with_decimals, decimals=2)

		if len(columns_with_integers) > 0:
			self._gt = self._gt.fmt_integer(columns_with_integers, use_seps=True, sep_mark=',')

		for value in self.formatted_data:
			self._gt = self._gt.tab_source_note(value)

		if self.type == 'concordance':
			if 'Left' in self.df.columns:
				self._gt = self._gt.cols_align(align='right', columns=['Left'])
			if 'Node' in self.df.columns:
				self._gt = self._gt.cols_align(align='center', columns=['Node'])


In [ ]:
#| export
@patch
def display(self: Result
			   ):
	""" Print analysis result output from conc in a nice table format using the great_tables library """
	
	self._prepare()
	self._gt.show()



In [ ]:
#| export
@patch
def to_html(self: Result,
			 collect_if_lazy: bool = True # if the df is a lazyframe, collect before returning
			 ):
	""" Return result output from conc as a dataframe """
	
	self._prepare()
	return self._gt.as_raw_html()

In [ ]:
result = Result(type = 'example', 
       df = pl.DataFrame({'Token': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
                          'Frequency': [5, 4, 3, 2, 1]}),
       title = 'Example Table',
       description = 'This is an example result.',
       summary_data = {},
       formatted_data = ['Formatted data text example 1', 'Formatted data text example 2']
      )

result.display()

Example Table 
 
 
 This is an example result. 
 
 
 Token 
 Frequency 
 
 
 
 
 Monday 
 5 
 
 
 Tuesday 
 4 
 
 
 Wednesday 
 3 
 
 
 Thursday 
 2 
 
 
 Friday 
 1 
 
 
 
 
 
 Formatted data text example 1 
 


 
 Formatted data text example 2

In [ ]:
result.to_frame()

Token,Frequency
str,i64
"""Monday""",5
"""Tuesday""",4
"""Wednesday""",3
"""Thursday""",2
"""Friday""",1


In [ ]:
#| hide
from IPython.display import display, HTML

In [ ]:
#| hide
display(HTML(result.to_html()))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()